# Testing USDA API connection

In [2]:
import pandas as pd
import requests
import json
import os
from pathlib import Path
from dotenv import load_dotenv

In [25]:
def fetch_yield(api_key:str, crop:list, start_year:str, end_year:str):
    """Fetches annual crop yield at the state level from USDA QuickStats."""

    base_url = "https://quickstats.nass.usda.gov/api/api_GET/?"

    params = {
        "key": api_key,
        "source_desc": "SURVEY",
        "sector_desc": "CROPS",
        "commodity_desc": crop,
        "statisticcat_desc": "YIELD",
        "agg_level_desc": "STATE",
        "unit_desc": "BU / ACRE",
        "year__GE": start_year,
        "year__LE":end_year,
        "format": "JSON"
    }
    response = requests.get(url=base_url, params=params)
    response.raise_for_status()

    return response.json()

In [28]:
load_dotenv()
API_KEY = str(os.getenv("API_KEY"))

data = fetch_yield(api_key=API_KEY, crop=["CORN", "SOYBEANS", "WHEAT"], start_year="2020", end_year="2025")

In [27]:
type(data)

dict

In [21]:

cwd = str(Path().resolve().parent)
file_name = "data/yield.json"
full_path = os.path.join(cwd,file_name)

try:
    with open(full_path, 'w') as f:
        json.dump(data, f, indent=4)
    print(f"Successfully exported dictionary to {file_name}")
except IOError as e:
    print(f"Error exporting file: {e}")

Successfully exported dictionary to data/yield.json


In [22]:
ls = list()

for d in data["data"]:
    ls.append(set(d.keys()))

print(ls) # list of sets

[{'county_ansi', 'group_desc', 'class_desc', 'util_practice_desc', 'domain_desc', 'statisticcat_desc', 'year', 'zip_5', 'commodity_desc', 'CV (%)', 'country_code', 'county_code', 'reference_period_desc', 'source_desc', 'end_code', 'country_name', 'week_ending', 'sector_desc', 'state_alpha', 'freq_desc', 'state_name', 'congr_district_code', 'Value', 'prodn_practice_desc', 'unit_desc', 'watershed_code', 'state_fips_code', 'short_desc', 'asd_code', 'domaincat_desc', 'location_desc', 'asd_desc', 'state_ansi', 'region_desc', 'watershed_desc', 'load_time', 'agg_level_desc', 'county_name', 'begin_code'}, {'county_ansi', 'group_desc', 'class_desc', 'begin_code', 'domain_desc', 'statisticcat_desc', 'util_practice_desc', 'year', 'zip_5', 'commodity_desc', 'CV (%)', 'country_code', 'county_code', 'reference_period_desc', 'source_desc', 'load_time', 'end_code', 'country_name', 'week_ending', 'sector_desc', 'state_alpha', 'freq_desc', 'congr_district_code', 'state_name', 'Value', 'unit_desc', 'wate

In [23]:
print(len(ls)) # 1004 sets in the list

3120


In [24]:
check = []
for k in ls:
    check.append(k == ls[0])

print(all(check))

True


In [14]:
# all dictionaries share the same keys.

In [15]:
data_filtered = [elem for elem in data["data"] 
                 if elem.get("reference_period_desc") == "YEAR" and elem.get("state_alpha") != "OT"]

len(data_filtered)


922

In [16]:

cwd = str(Path().resolve().parent)
file_name = "data/yield_filtered.json"
full_path = os.path.join(cwd,file_name)

try:
    with open(full_path, 'w') as f:
        json.dump(data_filtered, f, indent=4)
    print(f"Successfully exported dictionary to {file_name}")
except IOError as e:
    print(f"Error exporting file: {e}")

Successfully exported dictionary to data/yield_filtered.json


In [17]:
formatted_json = json.dumps(data_filtered, indent=4)
print(formatted_json)

[
    {
        "sector_desc": "CROPS",
        "location_desc": "ALABAMA",
        "prodn_practice_desc": "ALL PRODUCTION PRACTICES",
        "end_code": "00",
        "unit_desc": "BU / ACRE",
        "agg_level_desc": "STATE",
        "congr_district_code": "",
        "week_ending": "",
        "Value": "157",
        "source_desc": "SURVEY",
        "state_ansi": "01",
        "county_code": "",
        "statisticcat_desc": "YIELD",
        "country_code": "9000",
        "asd_code": "",
        "reference_period_desc": "YEAR",
        "domaincat_desc": "NOT SPECIFIED",
        "state_alpha": "AL",
        "country_name": "UNITED STATES",
        "state_fips_code": "01",
        "watershed_desc": "",
        "domain_desc": "TOTAL",
        "state_name": "ALABAMA",
        "county_name": "",
        "watershed_code": "00000000",
        "group_desc": "FIELD CROPS",
        "begin_code": "00",
        "year": 2025,
        "region_desc": "",
        "class_desc": "ALL CLASSES",
    

In [18]:
df = pd.DataFrame(data_filtered)[["year", "state_name", "commodity_desc","Value"]]
df

,year,state_name,commodity_desc,Value
0,2025,ALABAMA,CORN,157
1,2024,ALABAMA,CORN,112
2,2023,ALABAMA,CORN,164
3,2022,ALABAMA,CORN,118
4,2021,ALABAMA,CORN,163
...,...,...,...,...
917,2024,WYOMING,WHEAT,31
918,2023,WYOMING,WHEAT,30
919,2022,WYOMING,WHEAT,17
920,2021,WYOMING,WHEAT,32


# What was done?
-API call, early filering to avoid duplicates, and convertion to Data Frame.